In [ ]:
#definition of functions

import os
import glob
import SimpleITK as sitk
from skimage import exposure
from skimage import filters
from scipy.ndimage import morphology
from skimage import measure
from PIL import Image
import numpy as np
import pydicom
import tensorflow as tf
from keras import backend as K
from keras.backend import tensorflow_backend
from keras.models import load_model

def bgd_masking(img,thres=-300): #mask non-tissue area
    erode=morphology.binary_closing(morphology.binary_dilation(morphology.binary_erosion(img > thres)))    
    mask=morphology.binary_fill_holes(erode)
    blobs_labels = measure.label(mask, background=0)
    blob_hist, blob_bins_center = exposure.histogram(blobs_labels)
    first=np.argmax(blob_hist[1:])+1;
    maskimg=np.copy(img)
    maskimg[blobs_labels!=first]=np.min(maskimg)
    return maskimg

def dcmtonumpy3d(dcm_list,minimum=-160,maximum=240,new_y=160,new_x=160): #convert dcm to numpy (5mm slice)
    final_array = np.empty((0,new_y,new_x))
    dcm_ex = pydicom.dcmread(dcm_list[0])
    thickness = dcm_ex[0x0018, 0x0050].value
    interval = int(np.ceil(5.0 / thickness))
    slicenum_list = np.arange(0,len(dcm_list),interval)
    
    for slicenum in slicenum_list:
        dcm = pydicom.dcmread(dcm_list[slicenum])
        slice_array = dcm.pixel_array
        slice_array = bgd_masking(slice_array)
        rescaleslope = dcm.RescaleSlope
        rescaleintercept  = dcm.RescaleIntercept
        slice_array = slice_array * rescaleslope + rescaleintercept
        slice_array = np.clip(slice_array,minimum,maximum)
        slice_array = np.round((slice_array - minimum) * 255 / (maximum - minimum)).astype("int16")
        
        spacing = dcm[0x0028, 0x0030].value
        space = float(spacing[0])
        res = int(256 * space) #spacing 2mm       
        pil_data = Image.fromarray(slice_array)
        pil_resize = pil_data.resize(size=(res,res))
        slice_array = np.array(pil_resize)

        slice_y = slice_array.shape[0]
        slice_x = slice_array.shape[1]

        new_array = np.zeros((new_y,new_x))

        if slice_y >= new_y:
            crop_y = int(((slice_y - new_y) / 2))
            slice_array = slice_array[crop_y:crop_y+new_y,:]
            start_y = 0
            end_y = start_y + new_y
        else:
            start_y = int(np.floor((new_y - slice_y) / 2))
            end_y = start_y + slice_y

        if slice_x >= new_x:
            crop_x = int(np.floor((slice_x - new_x) / 2))
            slice_array = slice_array[:,crop_x:crop_x+new_x]
            start_x = 0
            end_x = start_x + new_x
        else:
            start_x = int(np.floor((new_x - slice_x) / 2))
            end_x = start_x + slice_x

        new_array[start_y:end_y,start_x:end_x] = slice_array
        new_array = new_array.reshape(1,new_y,new_x)
        final_array = np.concatenate([final_array, new_array])
    return final_array

def center_extraction(image): #estimation of the central point of the object
    shape_z = image.shape[0]
    shape_y = image.shape[1]
    shape_x = image.shape[2]
    outlist = []
    for i in range(shape_z):
        outsum = np.sum(image[i])
        outlist.append(outsum)
    center_z  = np.argmax(outlist)
    image_cz = image[center_z]
    ant_edge_list = []
    pos_edge_list = []
    
    y_flag = 0
    for j in range(shape_y): #large_y
        y_sum = np.sum(image_cz[j])
        if y_sum >= 2 and y_flag == 0:
            ant_edge_list.append(j)
            y_flag = 1
        elif y_sum < 2 and y_flag == 1:
            pos_edge_list.append(j)
            y_flag = 0
        if j == (shape_y - 1):
            y_flag = 0
    if len(ant_edge_list) == len(pos_edge_list) + 1:
        pos_edge_list.append(shape_y - 1)
    ant_edge_list = np.array(ant_edge_list)
    pos_edge_list = np.array(pos_edge_list)
    
    try:
        ydif = pos_edge_list - ant_edge_list
        center_y = int((ant_edge_list[np.argmax(ydif)] + pos_edge_list[np.argmax(ydif)]) / 2)

        right_edge_list = []
        left_edge_list = []

        x_flag = 0
        for k in range(shape_x): #half-largex
            if image_cz[center_y][k] >= 0.5 and x_flag == 0:
                right_edge_list.append(k)
                x_flag = 1
            elif image_cz[center_y][k] < 0.5 and x_flag == 1:
                left_edge_list.append(k)
                x_flag = 0
            if k == (shape_x - 1):
                x_flag = 0
        if len(right_edge_list) == len(left_edge_list) + 1:
            left_edge_list.append(shape_x - 1)        
        right_edge_list = np.array(right_edge_list)
        left_edge_list = np.array(left_edge_list)
        xdif = left_edge_list - right_edge_list
        center_x = int((right_edge_list[np.argmax(xdif)] + left_edge_list[np.argmax(xdif)]) / 2)
        return center_z, center_y, center_x
    except ValueError:
        return None, None, None

def crop3dimage(image,center_z,center_y,center_x,z_length,y_length,x_length):
    start_z = np.minimum(np.maximum(0, int(center_z - (z_length / 2))), int(image.shape[0] - z_length))
    start_y = np.minimum(np.maximum(0, int(center_y - (y_length / 2))), int(image.shape[1] - y_length))
    start_x = np.minimum(np.maximum(0, int(center_x - (x_length / 2))), int(image.shape[2] - x_length))
    croppedimage = image[start_z:start_z+z_length,start_y:start_y+y_length,start_x:start_x+x_length]
    return croppedimage
    
def bbox_edge_2d(array,image, width=12): #calculate the edge of the bounding box 
    shape_z = image.shape[0]
    shape_y = image.shape[1]
    shape_x = image.shape[2]
    outlist = []
    up_edge_list = []
    down_edge_list = []
    ant_edge_list = []
    pos_edge_list = []
    right_edge_list = []
    left_edge_list = []
    
    z_flag = 0
    for i in range(shape_z):
        outsum = np.sum(image[i])
        outlist.append(outsum)
        if outsum >= 2 and z_flag == 0:
            up_edge_list.append(i)
            z_flag = 1
        elif outsum < 2 and z_flag == 1:
            down_edge_list.append(i)
            z_flag = 0
        if i == (shape_z - 1):
            z_flag = 0
    if len(up_edge_list) == len(down_edge_list) + 1:
        down_edge_list.append(shape_z - 1)
        
    center_z = np.argmax(outlist)    
    image_cz = image[center_z]
    
    y_flag = 0
    for j in range(shape_y): 
        y_sum = np.sum(image_cz[j])
        if y_sum >= 1 and y_flag == 0:
            ant_edge_list.append(j)
            y_flag = 1
        elif y_sum < 1 and y_flag == 1:
            pos_edge_list.append(j)
            y_flag = 0
        if j == (shape_y - 1):
            y_flag = 0    
    if len(ant_edge_list) == len(pos_edge_list) + 1:
        pos_edge_list.append(shape_y - 1)
        
    ant_edge_list = np.array(ant_edge_list)
    pos_edge_list = np.array(pos_edge_list)
    ydif = pos_edge_list - ant_edge_list
    ant_edge = ant_edge_list[np.argmax(ydif)] 
    pos_edge = pos_edge_list[np.argmax(ydif)] 
    length_y = pos_edge - ant_edge
    center_y = int((ant_edge + pos_edge) / 2)
    
    x_flag = 0
    for k in range(shape_x): 
        if image_cz[center_y][k] >= 0.5 and x_flag == 0:
            right_edge_list.append(k)
            x_flag = 1
        elif image_cz[center_y][k] < 0.5 and x_flag == 1:
            left_edge_list.append(k)
            x_flag = 0
        if k == (shape_x - 1):
            x_flag = 0  
    if len(right_edge_list) == len(left_edge_list) + 1:
        left_edge_list.append(shape_x - 1)
        
    right_edge_list = np.array(right_edge_list)
    left_edge_list = np.array(left_edge_list)
    try:
        xdif = left_edge_list - right_edge_list
        right_edge = right_edge_list[np.argmax(xdif)] 
        left_edge = left_edge_list[np.argmax(xdif)] 
        length_x = left_edge - right_edge
        center_x = int((right_edge + left_edge) / 2)
        length = np.maximum(length_x, length_y)+ width

        final_right = np.maximum(center_x - int(length / 2), 0)
        final_left = np.minimum(center_x + int(length / 2), (shape_x - 1))
        final_ant = np.maximum(center_y - int(length / 2), 0)
        final_pos = np.minimum(center_y + int(length / 2), (shape_y - 1))  

        return center_z, final_ant, final_pos, final_right, final_left
    except ValueError:
        return None, None, None, None, None

def minus_dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (-2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

def loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    bce = tf.keras.losses.BinaryCrossentropy()
    return bce(y_true_f, y_pred_f) + 1 + minus_dice_coef(y_true, y_pred)

print("Definition finished")

In [ ]:
#parameter

#path of RCC cases(directory list)
cases = os.listdir("cases_example/")

#path of directory for saving images of extracted RCC
save_path = "save_example/"

#GPU
os.environ["CUDA_VISIBLE_DEVICES"]="2"
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)

#threshold
kidney_threshold = 1000 #threshold of kidney-segmented pixel count
RCC_threshold = 50 #threshold of RCC-segmented voxel count

#model
kidney_model = load_model("models/kidney_segmentation.h5", custom_objects={"minus_dice_coef":minus_dice_coef})
smallRCC_model = load_model("models/RCC_segmentation.h5",custom_objects={"loss":loss, "minus_dice_coef":minus_dice_coef})

print("Declaration is completed.")

In [ ]:
#model application

for case in cases:
    print(case)
    dicom_list = glob.glob("cases_example/" + case + "/*.dcm")
    dicom_list.sort()
    target = dcmtonumpy3d(dicom_list)
    z_shape = target.shape[0]
    if z_shape < 40: #zero padding if the number of slices is < 40
        new_target = np.zeros((40,160,160))
        s = int((40- target.shape[0]) / 2)
        new_target[s:s+z_shape] = target
        target = new_target

    right_kidney_seg = np.zeros((target.shape[0],target.shape[1],int(target.shape[2] / 2)))
    left_kidney_seg = np.zeros((target.shape[0],target.shape[1],int(target.shape[2] / 2)))

    for i in range(target.shape[0]):
        image = (target[i] / 255).astype("float32")
        img_input = image.reshape(-1,target.shape[1],target.shape[2],1)
        kid = kidney_model.predict(img_input)
        kid = kid.reshape(target.shape[1],target.shape[2])
        k_r = kid[:,:int(target.shape[2] / 2)]
        k_l = kid[:,int(target.shape[2] / 2):]
        right_kidney_seg[i] = k_r
        left_kidney_seg[i] = k_l

    # right RCC
    if np.sum(right_kidney_seg) <= kidney_threshold:
        print("Right Kidney Undetected")
    else:
        center_rz,center_ry,center_rx = center_extraction(right_kidney_seg)
        if center_rz is not None:
            right_cropped = crop3dimage(target[:,:,:int(target.shape[2] / 2)],center_rz,center_ry,center_rx,40,64,64)
            ckr = (right_cropped / 255).astype("float32")
            ckr_input = ckr.reshape(-1,40,64,64,1)
            rccseg_r = smallRCC_model.predict(ckr_input)

            if np.sum(rccseg_r) > RCC_threshold: 
                rccseg_r = rccseg_r.reshape(40,64,64)
                r_center, r_ant, r_pos, r_right, r_left = bbox_edge_2d(right_cropped,rccseg_r,12)
                if r_center is not None:
                    slice_r = right_cropped[r_center]
                    slice_r = slice_r[..., np.newaxis].astype("int16")
                    img_colr = np.concatenate([slice_r,slice_r,slice_r],axis=-1)
                    img_colr[r_ant,r_right:r_left,1:3] = 0
                    img_colr[r_pos,r_right:r_left,1:3] = 0
                    img_colr[r_ant:r_pos,r_right,1:3] = 0
                    img_colr[r_ant:r_pos+1,r_left,1:3] = 0
                    img_colr[r_ant,r_right:r_left,0] = 255
                    img_colr[r_pos,r_right:r_left,0] = 255
                    img_colr[r_ant:r_pos,r_right,0] = 255
                    img_colr[r_ant:r_pos+1,r_left,0] = 255
                    img_colr = img_colr.astype("uint8")
                    imgr = Image.fromarray(img_colr)
                    imgr.save(save_path + case + "_r.png") #save right RCC-suspected lesion as PNG file
                    print("Right RCC-susp lesion detected")
                else:
                    print("Right RCC-susp lesion extraction failed")                
            else:
                print("No right RCC")
        else:
            print("Right kidney extraction failed")
    
    #left RCC
    center_lz,center_ly,center_lx = center_extraction(left_kidney_seg)
    left_cropped = crop3dimage(target[:,:,80:],center_lz,center_ly,center_lx,40,64,64)

    if np.sum(left_kidney_seg) <= kidney_threshold:
        print("Left kidney undetected")
    else:
        center_lz,center_ly,center_lx = center_extraction(left_kidney_seg)
        if center_lz is not None:
            left_cropped = crop3dimage(target[:,:,int(target.shape[2] / 2):],center_lz,center_ly,center_lx,40,64,64)
            ckl = (left_cropped / 255).astype("float32")
            ckl_input = ckl.reshape(-1,40,64,64,1)
            rccseg_l = smallRCC_model.predict(ckl_input)

            if np.sum(rccseg_l) > RCC_threshold: 
                rccseg_l = rccseg_l.reshape(40,64,64)
                l_center, l_ant, l_pos, l_right, l_left = bbox_edge_2d(left_cropped,rccseg_l,12)
                if l_center is not None:
                    slice_l = left_cropped[l_center]
                    slice_l = slice_l[..., np.newaxis].astype("int16")
                    img_coll = np.concatenate([slice_l,slice_l,slice_l],axis=-1)
                    img_coll[l_ant,l_right:l_left,1:3] = 0
                    img_coll[l_pos,l_right:l_left,1:3] = 0
                    img_coll[l_ant:l_pos,l_right,1:3] = 0
                    img_coll[l_ant:l_pos+1,l_left,1:3] = 0
                    img_coll[l_ant,l_right:l_left,0] = 255
                    img_coll[l_pos,l_right:l_left,0] = 255
                    img_coll[l_ant:l_pos,l_right,0] = 255
                    img_coll[l_ant:l_pos+1,l_left,0] = 255
                    img_coll = img_coll.astype("uint8")
                    imgl = Image.fromarray(img_coll)
                    imgl.save(save_path + case + "_l.png") #save left RCC-suspected lesion as PNG file
                    print("Left RCC-susp lesion detected")
                else:
                    print("Left RCC-susp lesion extraction failed")                
            else:
                print("No left RCC")
        else:
            print("Left kidney extraction failed")
    print("")